# Make candidate coordinates file

Contact: Melissa Graham

Create a companion file to `all_nightly_epochs_files/candidate_lightcurve_parameters.dat` that lists the candidates RA and Declination in decimal degrees.

In [1]:
import psycopg2
import psycopg2.extras
import getpass
import pandas

import numpy as np
import time

Connect to the database and set the `tagid`.

In [2]:
dbuser = input("DB User: ")
dbpasswd = getpass.getpass("DB Password: ")
db = psycopg2.connect(f"dbname='decat' user='{dbuser}' password='{dbpasswd}' host='decatdb.lbl.gov'")
db.autocommit = True
cursor = db.cursor( cursor_factory = psycopg2.extras.DictCursor )

DB User:  decat_ro
DB Password:  ········


In [3]:
tag = "latest"
db.rollback()
q = ( "SELECT id, tag FROM versiontags WHERE tag=%(tag)s" )
cursor.execute( q, { "tag": tag } )
row = cursor.fetchone()
tagid = row['id']
print(tagid)

1


Retrieve all candidate identifiers and coordinates. This takes about 30 seconds.

In [4]:
%%time
q = ("SELECT id, ra, dec FROM candidates ")
cursor.execute(q, {'tagid': tagid})
df = pandas.DataFrame(cursor.fetchall())
print(len(df))

5162347
CPU times: user 13.9 s, sys: 1.57 s, total: 15.5 s
Wall time: 31.8 s


In [5]:
df

,0,1,2
0,DC21ldlr,7.056681,-43.817997
1,DC21mrrd,7.053866,-43.810593
2,DC21nehz,7.064672,-43.804777
3,DC21kxby,7.075488,-43.812880
4,DC21lwkt,7.091144,-43.809536
...,...,...,...
5162342,DC23lhzvc,271.616022,-29.284900
5162343,DC23lhzvx,270.871355,-29.618503
5162344,DC23lhzws,271.583515,-29.749514
5162345,DC23lhzxn,270.066982,-29.776693


Convert to `numpy` arrays to make matching faster. Delete `pandas` array.

In [6]:
cand_id  = np.asarray(df[0], dtype='str')
cand_ra  = np.asarray(df[1], dtype='float')
cand_dec = np.asarray(df[2], dtype='float')
del df

Get the list of candidate identifiers only for candidates with nightly-epoch light curves.

In [7]:
fnm = 'all_nightly_epochs_files/candidate_lightcurve_parameters.dat'
paramLC_candid = np.loadtxt(fnm, dtype='str', usecols=(1))
print(len(paramLC_candid))
del fnm

8998


In [8]:
paramLC_candid

array(['DC21baaa', 'DC21baab', 'DC21baae', ..., 'DC23lfqge', 'DC23lfrjt',
       'DC23lfrqm'], dtype='<U9')

Cross-match and create output file. This takes about 5 minutes.

In [9]:
t1 = time.time()

fnm = 'all_nightly_epochs_files/candidate_coordinates.dat'
fout = open(fnm, 'w')

fout.write('# Melissa Graham, create_candidate_coords_list.ipynb \n')
tnow = time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.gmtime())
fout.write('# UTC = '+tnow+' \n')
del tnow
fout.write('# \n')
fout.write('# Columns \n')
fout.write('#  0 id -- candidate identifier \n')
fout.write('#  1 Right Ascension [decimal degrees] \n')
fout.write('#  2 Declination [decimal degrees] \n')
fout.write('# \n')

for c, candid in enumerate(paramLC_candid):
    if (c == 2) | (c == 10) | (c == 100) | (c == 1000) | (c == 5000):
        t2 = time.time()
        print('%5i out of %5i  elapsed: %5.1f' % (c, len(paramLC_candid), (t2 - t1)))
    tx = np.where(cand_id == candid)[0]
    if len(tx) != 1:
        print('multiple matches  ', candid, tx)
    fout.write('%-20s %20.14f %20.14f \n' % (candid, cand_ra[tx[0]], cand_dec[tx[0]]))
    del tx
    
fout.close()
print('Wrote to: ',fnm)
del fnm

t2 = time.time()
print('elapsed: %5.1f' % (t2 - t1))
del t1, t2

    2 out of  8998  elapsed:   0.1
   10 out of  8998  elapsed:   0.4
  100 out of  8998  elapsed:   3.7
 1000 out of  8998  elapsed:  36.9
 5000 out of  8998  elapsed: 182.9
Wrote to:  all_nightly_epochs_files/candidate_coordinates.dat
elapsed: 325.7


In [ ]:
del paramLC_candid

As above, but for the PQLD candidates only.

In [7]:
fnm = 'all_nightly_epochs_files/PQLD_candidate_lightcurve_parameters.dat'
pqld_paramLC_candid = np.loadtxt(fnm, dtype='str', usecols=(1))
print(len(pqld_paramLC_candid))
del fnm

9814


In [8]:
t1 = time.time()

fnm = 'all_nightly_epochs_files/PQLD_candidate_coordinates.dat'
fout = open(fnm, 'w')

fout.write('# Melissa Graham, PQLD_create_candidate_coords_list.ipynb \n')
tnow = time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.gmtime())
fout.write('# UTC = '+tnow+' \n')
del tnow
fout.write('# \n')
fout.write('# Columns \n')
fout.write('#  0 id -- candidate identifier \n')
fout.write('#  1 Right Ascension [decimal degrees] \n')
fout.write('#  2 Declination [decimal degrees] \n')
fout.write('# \n')

for c, candid in enumerate(pqld_paramLC_candid):
    if (c == 2) | (c == 10) | (c == 100) | (c == 1000) | (c == 5000):
        t2 = time.time()
        print('%5i out of %5i  elapsed: %5.1f' % (c, len(pqld_paramLC_candid), (t2 - t1)))
    tx = np.where(cand_id == candid)[0]
    if len(tx) != 1:
        print('multiple matches  ', candid, tx)
    fout.write('%-20s %20.14f %20.14f \n' % (candid, cand_ra[tx[0]], cand_dec[tx[0]]))
    del tx
    
fout.close()
print('Wrote to: ',fnm)
del fnm

t2 = time.time()
print('elapsed: %5.1f' % (t2 - t1))
del t1, t2

    2 out of  9814  elapsed:   0.0
   10 out of  9814  elapsed:   0.2
  100 out of  9814  elapsed:   1.8
 1000 out of  9814  elapsed:  17.7
 5000 out of  9814  elapsed:  82.8
Wrote to:  all_nightly_epochs_files/PQLD_candidate_coordinates.dat
elapsed: 178.3
